In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
from sklearn.utils import check_random_state

# ignore warnings
import warnings
warnings.filterwarnings('ignore');

# display multiple outputs within a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all";

In [ ]:
def norm(x, axis=None):
    return np.linalg.norm(x, axis=axis)

def poincare_dist(u, v, eps=1e-5):
    d = 1 + 2 * norm(u-v)**2 / ((1 - norm(u)**2) * (1 - norm(v)**2) + eps)
    return np.arccosh(d)

In [ ]:
def generate_data(n):
    theta = np.random.uniform(0, 2*np.pi, n)
    u = np.random.uniform(0, 0.7, n)
    r = np.sqrt(u)
    x = r * np.cos(theta)
    y = r * np.sin(theta)
    init_data = np.hstack((x.reshape(-1,1), y.reshape(-1,1)))
    return init_data

def plot_poincare(data, title=None):
    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(111)
    circ = plt.Circle((0, 0), radius=1, edgecolor='black', facecolor='None', linewidth=3, alpha=0.5)
    ax.add_patch(circ)
    ax.scatter(data[:,0], data[:,1], s=20, alpha=1)
    if title != None: plt.title(title)
    plt.show()

n=10
data = generate_data(n)
delta = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        delta[i][j] = poincare_dist(data[i], data[j])
plot_poincare(data, title='Sample data')

def init_embed(n, dim=2, low=-0.1, high=0.1):
    random_config = np.random.uniform(low, high, size=(n, dim))
    return random_config

init_config = init_embed(10)
plot_poincare(init_config, title='Initial Configuration in Poincare Disk')